This is a tutorial to use the flip package: https://github.com/corentinravoux/flip \
It is self-contained and can be used in google collab or on your environement \
All the data used are subsampled version of a simulation. \
The data size is small for the tutorial, do not use it for science case. \

In [1]:
%%capture
!pip install git+https://github.com/corentinravoux/flip

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from flip import utils, plot_utils, data_vector, __flip_dir_path__
from flip.covariance import covariance, contraction, fitter
import flip
flip_base = Path(__flip_dir_path__)
data_path = flip_base / "data"
plt.style.use(data_path / "style.mplstyle")

Loading the data, located in the package itself



In [3]:
data_velocity = pd.read_parquet(data_path / "data_velocity.parquet")


data_velocity_true = {**data_velocity.rename(columns={'vpec': 'velocity'}).to_dict(orient='list'), 
                      "velocity_error": np.zeros(len(data_velocity["vpec"].values))}


# Create the data Vector
DataTrueVel = data_vector.DirectVel(data_velocity_true)

ktt, ptt = np.loadtxt(data_path / "power_spectrum_tt.txt")
kmt, pmt = np.loadtxt(data_path / "power_spectrum_mt.txt")
kmm, pmm = np.loadtxt(data_path / "power_spectrum_mm.txt")

sigmau_fiducial = 15

power_spectrum_dict = {"vv": [[ktt, ptt * utils.Du(ktt, sigmau_fiducial) ** 2]]}


/tmp/ipykernel_460407/2175734724.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  data_velocity_true = {**data_velocity.rename(columns={'vpec': 'velocity'}).to_dict(orient='list'),


# Fit true velocity interpolation sigma_u

In the model, there is a sigma_u parameter which can be interpreted to the smoothing of SNIa position due to peculiar velocities. \
To fit simultaneously this parameter, we interpolate the covariance matrix. 

In [4]:
sigmau_list = np.linspace(10.0, 20.0,20)[:, np.newaxis]
covariance_list = []
size_batch=10_000
number_worker=16


for sigu in sigmau_list:
    
    power_spectrum_dict = {"vv": [[ktt, ptt* utils.Du(ktt, sigu)**2]]}

    covariance_list.append(
        DataTrueVel.compute_covariance(
            "carreres23",
            power_spectrum_dict,
            size_batch=size_batch,
            number_worker=number_worker
        )
    )


/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
[ 000005.84 ]: 01-19 03:04  root            INFO     Covariance matrix generated from flip with carreres23 model in 7.60e-01 seconds
[ 000006.78 ]: 01-19 03:04  root            INFO     Covariance matrix generated from flip with carreres23 model in 9.11e-01 seconds
[ 000007.69 ]: 01-19 03:04  root            INFO     Covariance matrix generated from flip with carreres23 model in 9.04e-01 seconds
[ 000008.58 ]: 01-19 03:04  root            INFO     Covariance matrix generated from flip with carreres23 model in 8.80e-01 seconds
[ 000009.42 ]: 01-19 03:04  root            INFO     Covariance matrix generated from flip with carreres23 model in 8.24e-01 seconds
[ 000010.26 ]: 01-19 03:04  root

In [5]:
emulator_model = "skgpmatrix"
emulator_parameter_names = ["sigu"]

covariance_gpmatrix = covariance.CovMatrix.init_from_emulator(
            emulator_model,
            "velocity",
            covariance_list,
            sigmau_list,
            emulator_parameter_names,
            verbose=True,
            test_emulator=False,
        )



/global/homes/r/ravouxco/.local/perlmutter/python-3.11/lib/python3.11/site-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning: lbfgs failed to converge after 23 iteration(s) (status=2):
ABNORMAL: 

You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/global/homes/r/ravouxco/.local/perlmutter/python-3.11/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/global/homes/r/ravouxco/.local/perlmutter/python-3.11/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__constant_value is close to the specified upper bound 100000.0. Increasing the 

In [6]:
parameter_dict = {"fs8": {"value": 0.4,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                  "sigv": {"value": 200,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                  "sigu": {"value": 15.5,
                          "limit_low" : 10.0,
                          "limit_up" : 20.0,
                          "fixed" : False,
                         },
                 }


prior_dict = {"sigu": {"type": "gaussian",
                       "mean" : 15.0,
                       "standard_deviation" : 5.0,},
             }

likelihood_type="multivariate_gaussian"
likelihood_properties = {"inversion_method": "cholesky",
                         "prior": prior_dict}



minuit_fitter = fitter.FitMinuit.init_from_covariance(
        covariance_gpmatrix,
        DataTrueVel,
        parameter_dict,
        likelihood_type=likelihood_type,
        likelihood_properties = likelihood_properties,
)

print('Free parameters:', minuit_fitter.likelihood.free_par)

minuit_fitter.run()


Free parameters: ['sigv', 'fs8']


[ 000053.09 ]: 01-19 03:04  root            INFO     ┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 3781                       │              Nfcn = 99               │
│ EDM = 6.7e-05 (Goal: 0.0001)     │            time = 2.3 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│     SOME parameters at limit     │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────

{'fs8': 0.40509373127653925,
 'sigv': 324.9339590286237,
 'sigu': 10.000110261754214}